In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import pickle
import os
from pathlib import Path
from typing import Tuple, Dict
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm

In [ ]:
#K_VAL = 1

#INPUT_DIR = Path('Test_Train_Data')

TRAIN_PATH = "data_k1_train.txt"
TEST_PATH = "data_k1_test.txt"

In [ ]:
def load_data_and_create_matrices(  train_file: Path,
                                    test_file: Path ) -> Tuple[sp.csr_matrix, sp.csr_matrix, Dict[int, int], Dict[int, int]]:

    raw_data = []

    def parse_file(filepath: Path, dataset_type: str) -> None:
        if not filepath.exists():
            raise FileNotFoundError(f"File not found: {filepath}")

        with filepath.open("r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 2:
                    continue

                u_id = int(parts[0])
                items = [int(i) for i in parts[1:]]

                for i_id in items:
                    raw_data.append(
                        {
                            "user": u_id,
                            "item": i_id,
                            "type": dataset_type,
                        }
                    )

    parse_file(train_file, "train")
    parse_file(test_file, "test")

    df = pd.DataFrame(raw_data)
    print(f"Loaded {len(df):,} total interactions.")

    df["user_idx"] = df["user"].astype("category").cat.codes
    df["item_idx"] = df["item"].astype("category").cat.codes


    user_map: Dict[int, int] = dict(zip(df["user_idx"], df["user"]))
    item_map: Dict[int, int] = dict(zip(df["item_idx"], df["item"]))

    n_users = len(user_map)
    n_items = len(item_map)

    print(f"Matrix dimensions: {n_users:,} users x {n_items:,} items")

    def build_csr(dataset_type: str) -> sp.csr_matrix:
        subset = df[df["type"] == dataset_type]

        rows = subset["user_idx"].values
        cols = subset["item_idx"].values
        data = np.ones(len(subset), dtype=np.float32)

        return sp.csr_matrix((data, (rows, cols)), shape=(n_users, n_items))

    train_matrix = build_csr("train")
    test_matrix = build_csr("test")

    print(f"Train nnz: {train_matrix.nnz:,} | Test nnz: {test_matrix.nnz:,}")

    return train_matrix, test_matrix, user_map, item_map

In [ ]:
train_matrix, test_matrix, user_map, item_map = load_data_and_create_matrices(Path(TRAIN_PATH), Path(TEST_PATH))

Loaded 2,380,730 total interactions.
Matrix dimensions: 52,643 users x 91,599 items
Train nnz: 1,924,739 | Test nnz: 455,991


In [ ]:
n_users, n_items = train_matrix.shape

In [ ]:
# item_degree[i] = # of interactions for item i
item_degree = np.asarray(train_matrix.sum(axis=0)).ravel()
print("Item degrees shape:", item_degree.shape)

Item degrees shape: (91599,)


In [ ]:
from pathlib import Path
from collections import defaultdict

cooccur = defaultdict(int)

print("Building item-item co-occurrence map...")
for u in tqdm(range(n_users), desc="Users"):
    items = train_matrix[u].indices
    if len(items) < 2:
        continue

    items = np.sort(items)
    for i in range(len(items)):
        for j in range(i + 1, len(items)):
            a, b = items[i], items[j]
            cooccur[(a, b)] += 1

print("Number of item pairs with nonzero co-occurrence:", len(cooccur))

Building item-item co-occurrence map...


Users: 100%|██████████| 52643/52643 [02:10<00:00, 404.71it/s] 

Number of item pairs with nonzero co-occurrence: 113008636


In [ ]:
K_NEIGHBORS = 5  # you can tune this

neighbors_tmp = [[] for _ in range(n_items)]

print("Computing Jaccard similarities from co-occurrence...")
for (a, b), inter in tqdm(cooccur.items(), desc="Pairs"):
    da = item_degree[a]
    db = item_degree[b]
    union = da + db - inter
    if union <= 0:
        continue
    sim = inter / union
    if sim <= 0:
        continue

    neighbors_tmp[a].append((b, sim))
    neighbors_tmp[b].append((a, sim))

neighbors_idx = np.full((n_items, K_NEIGHBORS), -1, dtype=np.int32)
neighbors_sim = np.zeros((n_items, K_NEIGHBORS), dtype=np.float32)

for i in range(n_items):
    sims = neighbors_tmp[i]
    if not sims:
        continue
    sims.sort(key=lambda x: x[1], reverse=True)
    sims = sims[:K_NEIGHBORS]

    for k, (j, s) in enumerate(sims):
        neighbors_idx[i, k] = j
        neighbors_sim[i, k] = s

print("neighbors_idx shape:", neighbors_idx.shape)
print("neighbors_sim shape:", neighbors_sim.shape)


Computing Jaccard similarities from co-occurrence...


Pairs: 100%|██████████| 113008636/113008636 [02:16<00:00, 826527.84it/s]


neighbors_idx shape: (91599, 5)
neighbors_sim shape: (91599, 5)


In [ ]:
def ndcg_at_k(r, k):
    r = np.asarray(r, dtype=float)[:k]
    if r.size == 0:
        return 0.0
    discounts = np.log2(np.arange(2, r.size + 2))
    dcg = np.sum(r / discounts)
    ideal = np.sort(r)[::-1]
    idcg = np.sum(ideal / discounts)
    return dcg / idcg if idcg > 0 else 0.0


In [ ]:
def recommend_user_jaccard(
    u_idx,
    train_matrix,
    neighbors_idx,
    neighbors_sim,
    topn=20
):
    user_row = train_matrix[u_idx].toarray().ravel()
    seen = user_row.nonzero()[0]

    if len(seen) == 0:
        return np.array([], dtype=int), np.array([])

    scores = np.zeros(train_matrix.shape[1], dtype=np.float32)

    for it in seen:
        nbrs = neighbors_idx[it]
        sims = neighbors_sim[it]

        mask = nbrs >= 0
        nbrs = nbrs[mask]
        sims = sims[mask]

        scores[nbrs] += sims

    scores[seen] = -np.inf

    valid = np.where(np.isfinite(scores))[0]
    if len(valid) == 0:
        return np.array([], dtype=int), np.array([])

    top_idx = np.argpartition(scores[valid], -topn)[-topn:]
    top_items = valid[top_idx]
    top_items = top_items[np.argsort(scores[top_items])[::-1]]

    return top_items, scores[top_items]


In [ ]:
test_items_per_user = [test_matrix[u].indices for u in range(n_users)]
print("Prepared test items per user.")

Prepared test items per user.


In [ ]:
TOPK = 20

EVAL_USER_LIMIT = None

all_user_indices = np.arange(n_users)
if EVAL_USER_LIMIT is not None:
    all_user_indices = all_user_indices[:EVAL_USER_LIMIT]

print(f"Evaluating {len(all_user_indices)} users...")

ndcg_list = []
predictions_for_save = []

processed = 0

for u_idx in tqdm(all_user_indices, desc="Predicting (Jaccard KNN)"):
    true_items = test_items_per_user[u_idx]
    if len(true_items) == 0:
        continue

    rec_items, _ = recommend_user_jaccard(
        u_idx=u_idx,
        train_matrix=train_matrix,
        neighbors_idx=neighbors_idx,
        neighbors_sim=neighbors_sim,
        topn=TOPK
    )

    orig_user = user_map[u_idx]
    orig_items = [item_map[i] for i in rec_items]
    line = " ".join([str(orig_user)] + [str(i) for i in orig_items])
    predictions_for_save.append(line)

    # ndcg
    relevance = [1 if i in true_items else 0 for i in rec_items]
    ndcg_val = ndcg_at_k(relevance, TOPK)
    ndcg_list.append(ndcg_val)

    processed += 1
    if processed % 5000 == 0:
        print(f"Processed {processed} users. Current Avg NDCG@{TOPK}: {np.mean(ndcg_list):.4f}")

avg_ndcg = np.mean(ndcg_list) if ndcg_list else 0.0
print("\nFINAL RESULT (Global Jaccard Item-KNN):")
print(f"NDCG@{TOPK}: {avg_ndcg:.4f}")

filename = f"jaccard_knn_k{K_NEIGHBORS}_top{TOPK}.txt"
with open(filename, "w") as f:
    for row in predictions_for_save:
        f.write(row + "\n")

print("Saved predictions to:", filename)


Evaluating 52643 users...


Predicting (Jaccard KNN):  10%|▉         | 5025/52643 [00:22<03:34, 221.77it/s]

Processed 5000 users. Current Avg NDCG@20: 0.2417


Predicting (Jaccard KNN):  19%|█▉        | 10022/52643 [00:44<03:06, 227.99it/s]

Processed 10000 users. Current Avg NDCG@20: 0.2328


...

Processed 45000 users. Current Avg NDCG@20: 0.2893


Predicting (Jaccard KNN):  95%|█████████▌| 50040/52643 [03:38<00:11, 228.99it/s]

Processed 50000 users. Current Avg NDCG@20: 0.2975


Predicting (Jaccard KNN): 100%|██████████| 52643/52643 [03:49<00:00, 229.26it/s]



FINAL RESULT (Global Jaccard Item-KNN):
NDCG@20: 0.2999
Saved predictions to: jaccard_knn_k5_top20.txt
